In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import webbrowser

In [25]:
df_for_32 = pd.DataFrame()
with open("C:\\Users\\dmona\\Downloads\\tuturu.txt", "r", encoding="utf-8") as file:
    for j, line in enumerate(file, start=0):
        soup = BeautifulSoup(line, "html.parser")
        data = {}
        i = 1
        spans = soup.find_all("span")
        len_spans = len(spans)
        if spans[3].text.strip() == 'прямой':
            if len_spans == 29:
                for span in spans:
                    if i != 12:
                        data[f"Column_{i}"] = [span.text.strip()]
                    else:
                        data[f"Column_{12}"] = ['']
                        data[f"Column_{13}"] = ['']
                        data[f"Column_{14}"] = ['']
                        data[f"Column_{15}"] = ['']
                        i += 3
                    i += 1
            if len_spans == 30:
                for span in spans:
                    if i != 13:
                        data[f"Column_{i}"] = [span.text.strip()]
                    else:
                        data[f"Column_{13}"] = ['']
                        data[f"Column_{14}"] = ['']
                        data[f"Column_{15}"] = ['']
                        i += 2
                    i += 1
            if len_spans == 31:
                for span in spans:
                    if span.text.strip() != '+1':
                        if i != 12:
                            data[f"Column_{i}"] = [span.text.strip()]
                        elif i == 12:
                            data[f"Column_{12}"] = ['']
                            data[f"Column_{13}"] = ['']
                            data[f"Column_{14}"] = ['']
                            data[f"Column_{15}"] = ['']
                            i += 3
                        i += 1
            if len_spans == 32:
                if spans[22].text.strip() == '+1':
                    for span in spans:
                        if span.text.strip() != '+1':
                            if i != 13:
                                data[f"Column_{i}"] = [span.text.strip()]
                            elif i == 13:
                                data[f"Column_{13}"] = ['']
                                data[f"Column_{14}"] = ['']
                                data[f"Column_{15}"] = ['']
                                i += 2
                            i += 1
                else:
                    for span in spans:
                        data[f"Column_{i}"] = [span.text.strip()]
                        i+= 1
            if len_spans == 33:
                for span in spans:
                    if i < 15:
                        data[f"Column_{i}"] = [span.text.strip()]
                    if i > 15:
                        data[f"Column_{i-1}"] = [span.text.strip()]
                    i += 1
            if len_spans == 34:
                for span in spans:
                    if i < 24:
                        data[f"Column_{i}"] = [span.text.strip()]
                    if i > 25:
                        data[f"Column_{i-2}"] = [span.text.strip()]
                    i += 1
            if len_spans == 35:
                    for span in spans:
                        if i < 10:
                            data[f"Column_{i}"] = [span.text.strip()]
                        if i > 10 and i < 25:
                            data[f"Column_{i-1}"] = [span.text.strip()]
                        if i > 26:
                            data[f"Column_{i-3}"] = [span.text.strip()]
                        i += 1
            new_row = pd.DataFrame(data)
            df_for_32 = pd.concat([df_for_32, new_row], ignore_index=True)
html_table = df_for_32.to_html()

# Записываем HTML-таблицу в файл
with open("df_table.html", "w", encoding="utf-8") as file:
    file.write(html_table)

# Открываем файл в веб-браузере
webbrowser.open("df_table.html")



True

In [26]:
final_df = pd.DataFrame()

final_df['Маршрут'] = df_for_32['Column_1']
final_df['Время в пути'] = df_for_32['Column_6']
final_df['Авиакомпания'] = df_for_32['Column_9']
final_df['Рейс'] = df_for_32['Column_11']
final_df['Модель самолета'] = df_for_32['Column_13']
final_df['Время вылета'] = df_for_32['Column_18']
final_df['Дата вылета'] = df_for_32['Column_19']
final_df['Откуда'] = df_for_32['Column_20']
final_df['Время прилета'] = df_for_32['Column_22']
final_df['Дата прилета'] = df_for_32['Column_23']
final_df['Куда'] = df_for_32['Column_24']
final_df['Цена билета'] = df_for_32['Column_27']
final_df['Ручная кладь'] = df_for_32['Column_29']
final_df['Багаж'] = df_for_32['Column_30']
final_df['Возврат'] = df_for_32['Column_31']
final_df['Обмен'] = df_for_32['Column_32']

In [27]:
html_table = final_df.to_html()

# Записываем HTML-таблицу в файл
with open("df_table.html", "w", encoding="utf-8") as file:
    file.write(html_table)

# Открываем файл в веб-браузере
webbrowser.open("df_table.html")

True